# Chapter 3. Dictionaries and Sets

```
Any running Python program has many dictionaries active at the same time, even if the user's program code doesn't explicity use a dictionary.

-- A. M. Kuching
```

The dict type is not only widely used in our programs but also a fundamental part of the Python implementation. Module namespaces, class and instance attributes, and function keyword arguemnts are some fundamental constructs where dictionaries are deployed. The built-in functions lives in `__builtins__.__dict__`

**Hash tabels are the engines behind Python's high-performance dicts.**

## Generic Mapping Types

The `collections.abc` module provides the Mapping and MutableMapping in ABCs to formalize the interfaces of dict and similary types.

![](http://processon.com/chart_image/5dd3ba7fe4b001c03aed4d7f.png)

All mapping types in the standard library use the basic dict in their implementationo, so they share the limitation that the keys must be hashable.

**What is Hashable?**

Here is part of definition of hashable:

    An object is hashable if it has a hash value which never change during its lifetime(it needs a `__hash__` method), and can be compared to other objects (it needs an `__eq__` method). Hashable objects which compare equal must have the same hash value.

The atomic immutable types(str, bytes, numeric types) are all hashable. A fronzenset is always hashable, because its elements must be hashable by definition. A tuple is hashable only if all its items are hashable.

In [3]:
tt = (1, 2, (30, 40))
hash(tt)

8027212646858338501

In [4]:
tl = (1, 2, [30, 40])
hash(tl)

TypeError: unhashable type: 'list'

In [5]:
tf = (1, 2, frozenset([30, 40]))
hash(tf)

985328935373711578

User-defined types are hashable by default because their hash value is their id() and they all compare not equal. If an oject implements a custom `__eq__` that takes into account its internal state, it may be hashable only if all its attributes are immutable.

## dict Comprehensions

Since Python 2.7, the syntax of listcomps and genexps was applied to dict comprehension. A dictcomp builds a dict instance by providing `key:value` pair from any iterable.

In [3]:
{ char:num for char,num in zip(list('ABCD'), range(4))}

{'A': 0, 'B': 1, 'C': 2, 'D': 3}

## Overview of Common Mapping Methods

The basic API for mappings is quite rich. Two of most useful variations: `defaultdict` and `OrderedDict`, both defined in the `collections` module.

### Handling Missing Keys with setdefault

In [8]:
my_dict = {}

if 'key' not in my_dict:
    my_dict['key'] = []
my_dict['key'].append('value')

replace with:

In [6]:
my_dict = {}
my_dict.setdefault('key', []).append('value')

## Mappings with Flexible Key Lookup

Sometimes it is convenient to have mappings that return some made-up value when a missing key is searched. There are two main approaches to this: one is to use a `defaultdict` instead of a plain dict. The other is to subclass dict or any other mapping type and add a `__missing__` method.

### defaultdict: Another Take on Missing Keys

`collections.defeaultdict` provides elegant solution to the problem. A defaultdict is configured to create items on demand whenever a missing key is searched.

Here is how it works: when instantiating a defaultdict, you provide a callable that is used to produce a default value whenever `__getitem__` is passed a nonexistent key For example, given a empty defauldict ceated as `dd = defaultdict(list)`, if `new-key` is not in dd, the expression `dd['new-key']` does the following steps:

1. Call list() to create a new list.
2. Inserts the list to dd using `new-key` as key.
3. Returns a reference to that list.

The callable that produce the default values is held in an instance attribute called `default_factory`.

In [9]:
import collections

In [10]:
dd = collections.defaultdict(list)
dd['key']

[]

In [11]:
dd.default_factory

list

In [12]:
help(dd)

Help on defaultdict object:

class defaultdict(builtins.dict)
 |  defaultdict(default_factory[, ...]) --> dict with default factory
 |  
 |  The default factory is called without arguments to produce
 |  a new value when a key is not present, in __getitem__ only.
 |  A defaultdict compares equal to a dict with the same items.
 |  All remaining arguments are treated the same as if they were
 |  passed to the dict constructor, including keyword arguments.
 |  
 |  Method resolution order:
 |      defaultdict
 |      builtins.dict
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __copy__(...)
 |      D.copy() -> a shallow copy of D.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __missing__(...)
 |      __missing__(key) # Called by __getitem__ for missing key; pseudo-code:
 |      if self.default_factory is None: raise Key

> The `default_factory` of a defaultdict is only invoked to provide default values for `__getitem__` calls, and not for the other methods. For example, if dd is defaultdict, and k is a missing key, `dd[k]` will call the default_factory to create a default value, but `dd.get(k)` still return None.

The mechanism that makes `default_factory` work by calling `default_factory` is actually the `__missing__` special method, a feature supported by all standard mapping types.

### The `__missing__` Method

Underlying the way mappings deal with missing keys is the aptly named `__missing__` method. This mothod is not defined in the base dict class, but dict is aware of it: if you subclass dict and provide a `__missing__` method, the standard `dict.__getitem__` will call it whenever a key is not found, instead of raising KeyError.

> The `__missing__` method is just called by `__getitem__`. The presence of a `__missing__` method has no effect on the behavior of other methods that look up keys, such as get or `__contains__`. This is why the `default_factory` of defaultdict works on with `__getitem__`.

## Variations of dict

`collections.OrderDict`
    
Maintains keys in insertion order, allowing iteration over items in a predictable order. The popitem method of an OrderDict pops the first item by default, but if called as `my_odict.popitem(last=True)`, it pops the last item added.

`collections.ChainMap`

Holds a list of mappings that can be searchd as one. The lookup is performed on each mapping in order, and succeeds if the key is found in any of them. This is useful to interpretes for languages with nested scopes, where each mapping represents a scope context.

In [1]:
import collections

In [2]:
import builtins

pylookup = collections.ChainMap(locals(), globals(), vars(builtins))

In [6]:
# globals()

`collections.Counter`

A mapping that holds an integer count for each key. Updating an existing key adds to its count. This can be used to count instances of hashable objects or as a multiset - a set that can hold several occurrences of each element.

In [8]:
ct = collections.Counter('abfesfesafel')
ct

Counter({'a': 2, 'b': 1, 'f': 3, 'e': 3, 's': 2, 'l': 1})

In [9]:
ct.update('aaabbbcccddd')
ct

Counter({'a': 5, 'b': 4, 'f': 3, 'e': 3, 's': 2, 'l': 1, 'c': 3, 'd': 3})

In [12]:
ct.most_common(3)

[('a', 5), ('b', 4), ('f', 3)]

`collections.UserDict`

A pure Python implementation of a mapping that works like a standard dict.

## Subclassing UserDict

It's almost easier to crate a new mapping type by extending UserDict rather that dict.

**The main reason why it's preferable to subclass from UserDict rather than from dict is that the built-in has some implementation shortcuts that end up forcing us to override methods that we can just inherit from UserDict with no problem.**

Note that `UserDict` does not inherit from dict, but has an internal dict instance, called data, which holds the actual items. This avoids undesired recursion when coding special methods like `__setitem__`, and simplifies the coding of `__contains__`.

In [13]:
import collections

In [14]:
class StrKeyDict(collections.UserDict):
    
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):
        return str(key) in self.data
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item

`MutableMapping.update`

This powerful method can be called directly but is also used by `__init__` to load the instance from other mappings from iterables of (key, value) paris and keyword arguments.

## Immutable Mappings

**Since Python 3.3, the `types` module provides a wrapper class called `MappingProxyType`, which given a mapping, returns a mappingproxy instance that is a read-only but dynamic view of the original mapping.**

In [16]:
from types import MappingProxyType

In [17]:
d = {1: 'A'}
d_proxy = MappingProxyType(d)

In [18]:
d_proxy[1]

'A'

In [19]:
d_proxy[2] = 'B'

TypeError: 'mappingproxy' object does not support item assignment

In [20]:
d[2] = 'B'

In [21]:
d_proxy

mappingproxy({1: 'A', 2: 'B'})

## Set Theory

A set is a collection of unique objects. A basic use case is removing duplications.

In [22]:
l = ['spam', 'spam', 'eggs', 'spam']
set(l)

{'eggs', 'spam'}

Set elements must be hashable. The set type is not hashable, but frozenset is, so you can have frozenset elements inside a set.

### set Literals

**The syntax of set literals -- `{1}, {1, 2}` -- looks exactly likes that math notations, with one important exception: there's no literals notation for the empty set, so we must remember to write set().**

>Don't forget: to create an empty set, you  should use the constructor without an argument: `set()`. If you write {}, you're creating an empty dict.

**Literal set syntax like `{1, 2, 3}` is both faster and more readable than calling constructor (e.g., `set([1, 2, 3)`). The latter form is slower because, to evaluate it, Python has to look up the set name to fetch the constructor, then build a list, and finally pass it to the constructor. In constract to, to process a literal like `{1, 2, 3}` Python run a speciallized BUILD_SET bytecode.**

In [25]:
import dis

dis.dis('{1}')

  1           0 LOAD_CONST               0 (1)
              2 BUILD_SET                1
              4 RETURN_VALUE


In [26]:
dis.dis('set([1])')

  1           0 LOAD_NAME                0 (set)
              2 LOAD_CONST               0 (1)
              4 BUILD_LIST               1
              6 CALL_FUNCTION            1
              8 RETURN_VALUE


### Set Comprehensions

Set comprehensions(setcomps) were added in Python 2.7.

In [29]:
from unicodedata import name

{chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i), '')}

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

### Set Operations

![](https://processon.com/chart_image/5dd3ba57e4b0096e8c0ff949.png)

## dict and set Under the Hood

Understanding how Python dictionaries and set are implemented using hash table is helpful to make sence of their strength and limitations.

Here are some questions are Python dict and set?

* How effiecient are Python dict and set?
* Why are they unordered?
* Why can't we use any Python oject as a dict key or set element?
* Why does the order of the dict keys or set elements depend on insertion order, and may change during the lifetime of the structure?
* Why is it hard to add items to a dict or set while literating through it?

### A Performance Experiment

### Hash Tables in Dictionaries

This is a high-level view of how Python uses a hash table to implement a dict. Many details are ommited, the CPython code has some optimization tricks, but the overall description is accurate.

> To simplify the ensuring presentation, we will focus on the internals of dict firset, and later transfer the concepts to sets.

A hash table is a sparse array(i.e., an array that always has empty cells). In standard data structure texts, the cells in a hash table called "buckets". In a dict hash table, there is a bucket for each item, and it contains two fields: a reference to the key and a reference to the value of item. Because all buckets have the same size, access to an individual bucket is done by offset.

Python tries to keep at least 1/3 of the bucket empty; if the hash table becomes too crowded, it is copied to a new location with room for more buckets.

To put an item in a hash table, the first step is to calculate the hash value of the item key, which is done with the `hash()` built-in function.

#### Hashes and equality

The `hash()` built-in function works directly with built-in types and falls back to calling `__hash__` for user-defined types. If two objects compare equal, their hash values must also be equal, otherwise the hash table algorithm does not work. For example, because `1 ==1.0 ` is True, `hash(1) == hash(1.0)` must also be true, even though the internal representation of an int and a float are every different.

To effective as hash table indexes, hash valus shold scatter around the index space as much as possible. This means objects are similar but not equal should have hash values that differ widely.

#### The hash table algorithm

To fetch the value at `my_dict[search_key]`, Python calls `hash(search_key)` to obtain the hash value of search_key and uses the least significant bits of the number as an offset to look up a bucket in the hash table(the number of bits used depends on the current size of the table). If the found bucket is empty, `KeyError` is raised. Otherwise, the found bucket has an item, a found_key:found_value paire, and then Python checks whether `search_key == found_key`. If they match, that was the item sought: found_value is returned.

**However, if seach_key and found_key do not match, this is a hash collision.** This happends because a hash function maps arbitrary object to a small number of bits, the hash table is indexed with a subset of those bits. In order to resolve the collision, the algorithm then takes different bits in the hash, messages them ina particular way, and uses the result as an offset to look up a different bucket. If that is empty, `KeyError` is raised; if not, either  the keys match and the item value is returned, or the collision resolution process is repeated.

![](http://processon.com/chart_image/5dd3ba57e4b0096e8c0ff949.png)

The process to insert or update an item is the same, except that when an empty bucket is located, the new item is put there, and when a bucket with a matching key is found, the value in that bucket is overwritten with the new value.

When inserting items, Python may detemine that the hash table is to crowed and rebuild it to a new location with more room. As the hash table grows, so does the number of hah bits used as bucket offsets, and this keeps the reate of collisions low.

This implementation may seem like a lot of work, but even with millions of items in a dict, many searches happens with no collisions, and the average number of collisions per search is between one and two.

### Practical Consequences of How dict Works

#### Keys must be hashable objects

An object is hashable if all of these requirements are met:

1. It supports the `hash()` function via a hash() method that always returns the same value of the lifetime of the object.
2. It supports equality via an `eq()` method.
3. if `a == b` is true then `hash(a) == hash(b)` must also be true.

User-defined types are hashable by deafult because their hash value is their `id()` and they all compare not equal.

> If you implement a class with a custom `__eq__` method, you must also implement a suitable `__hash__`, because you must always make sure if `a == b` is true then `hash(a) == hash(b)` is also be true.

#### dicts have significant memory overhead

Because a dict uses a hash table internally, and hash types must be sparse to work, they are not space efficient. For example, if you are handling a large quantity of records, if make sence to store them in a list of tuples or named tuples instead using a list of dictionaries in JSON style, with one dict per record. **Replacing dicts with tuples reduces the memory usage in two ways: by removing the overhead of one hash table per record and by not storing the field names again with each record.**

**Keep in mind we are talking about space optimization. If you are dealing with a few million objects and your computer has gigabytes of RAM, you should postpone such optimization until they are actually warranted. Optimization is the altar where maintainability is sacrificed.**

#### Key search is very fast

**The dict implementation is an example of trading space for time: dictionaries have significant memory overhead, but they provide fast access regardless of the size of the dictionary, as long as it fits in memory.**

#### Key ordering depends on insertion order

When a hash collision happens, the second key ends up in a position that it would not normally occupy if it had been inserted first.

In [3]:
DIAL_CODES = [
(86, 'China'),
        (91, 'India'),
        (1, 'United States'),
        (62, 'Indonesia'),
        (55, 'Brazil'),
        (92, 'Pakistan'),
        (880, 'Bangladesh'),
        (234, 'Nigeria'),
        (7, 'Russia'),
        (81, 'Japan'),
]

In [4]:
d1 = dict(DIAL_CODES)
d1.keys()

dict_keys([86, 91, 1, 62, 55, 92, 880, 234, 7, 81])

In [5]:
d2 = dict(sorted(DIAL_CODES))
d2.keys()

dict_keys([1, 7, 55, 62, 81, 86, 91, 92, 234, 880])

#### Adding items to a dict may change the order of existing keys

Whenever you add a new item to a dict, the Python interpreter may decide that the hash table of that dictionary needs to grow. This entails building a new bigger hash table, and adding all current items to the new table. During this process, new hash collision may happen, with the result that keys are likely to be ordered differently in the new hash table.

**This is why modifying the contents of a dict while iterating through it is a bad idea. If you need to scan and add items to a dictionary, do it in two steps: read the dict from start to finish and collect the needed additions in a second dict. Then update the first one with it.**

### How Set Work - Pratical Consequences

The set and frozensets types also implemented with a hash table, except that each bucket holds only a reference to the element.

* Set elements must be hashable objects.
* Sets have a significant memory overhead.
* Membership testing is very efficient.
* Element ordering depends on the insertion order.
* Adding elements to set may change the order of other elements.

## Chapter Summary

Dictionaries are a keystone of Python. Beyond the basic dict, the standard library offers handy, ready-to-use specialized mappings like `defaultdict`, `OrderDict`, `Counter`,  and `ChinaMap`, all defined in the collections module. The same module also provides the easy-to-extend UserDcit class.

Two powerful methods in available in most mappings are `setdefault` and `update`. The default method is used update items holding mutable values to avoid redundant searches for the same key. The `update` method allows bulk insertion or overwritting of items from any other mapping.

**A clever hook in the mapping API is the `__missing__` method, which lets you customize what happens when a key is not found.**

The `collections.abc` moudle provides the `Mapping` and `MutableMapping` abstract base classes for reference and type checking. The little-known `MappingProxyType` from the types moudle creates immutable mappings.

The hash table implementation underlying dict and set is extremely fast. Understanding its logic explains why items are unordered and may even reordered behind our backs.